Get the data and save it locally to drive

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# # We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# # save locally to drive
# !cp "input.txt" "/content/drive/MyDrive/Data_Science/LLM-from-scratch"
import os
os.getcwd()

Mounted at /content/drive


'/content'

In [3]:
# load in the tiny shakespeare dataset from txt file
with open('/content/drive/MyDrive/Data_Science/LLM-from-scratch/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
# these characters will be used in the LLM
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
# encode the characters
# create a mapping from characters to integers
encode_map = { character:index for index,character in enumerate(characters) }
decode_map = { index:character for index,character in enumerate(characters) }
encode = lambda s: [encode_map[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([decode_map[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

# There are many different kinds of encoders (see tiktoken, sentencepiece)
# usually there are large codebooks used at industrial level since compute is higher
# since this is a simple transformer model, a simpler encoder will suffice

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [5]:
# load data into a pytorch tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [6]:
# split into training and validation
n = int(0.9*len(data)) # 90/10 split
train_data = data[:n] # 90% of the data
val_data = data[n:] # 10% of the data

In [7]:
block_size = 8 # block size is the sample of data put into the network
train_data[:block_size+1] # block_size cases of sequential relational relationships

# example of how inputs are put into network using sequential patterns to predict the target
# variation is good to train the network to find short and long patterns
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [8]:
torch.manual_seed(1337) # in order to replicate the results from the video
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # batch_size random ints in a 1D tensor
    x = torch.stack([data[i:i+block_size] for i in ix]) # get the random lines from the input.txt file
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # shape (B*T, C)
            targets = targets.view(B*T) # shape (B*T, 1)
            loss = F.cross_entropy(logits, targets) # negative log liklihood, expected loss is -ln(1/65)~=4.17

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [10]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [11]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.5727508068084717


In [12]:
# getting something better from before, but simple bigram model can't make good results.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y helti


In [13]:
!python '/content/drive/MyDrive/Data_Science/LLM-from-scratch/bigram.py'

step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 2.8110, val loss 2.8249
step 600: train loss 2.5434, val loss 2.5682
step 900: train loss 2.4932, val loss 2.5088
step 1200: train loss 2.4863, val loss 2.5035
step 1500: train loss 2.4665, val loss 2.4921
step 1800: train loss 2.4683, val loss 2.4936
step 2100: train loss 2.4696, val loss 2.4846
step 2400: train loss 2.4638, val loss 2.4879
step 2700: train loss 2.4738, val loss 2.4911

od nos CAy go ghanoray t, co haringoudrou clethe k,LARof fr werar,
Is fa!


Thilemel cia h hmboomyorarifrcitheviPO, tle dst f qur'dig t cof boddo y t o ar pileas h mo wierl t,
S:
STENENEat I athe thounomy tinrent distesisanimald 3I: eliento ald, avaviconofrisist me Busarend un'soto vat s k,
SBRI he the f wendleindd t acoe ts ansu, thy ppr h.QULY:
KIIsqu pr odEd ch,
APrnes ouse bll owhored miner t ooon'stoume bupromo! fifoveghind hiarnge s.
MI aswimy or m, wardd tw'To tee abifewoetsphin sed The a


## The mathematical trick in self-attention (historical running average)

In [14]:
# setting up variables
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x=torch.randn(B,T,C)

# version 1: calculating running average with for loops
xbow = torch.zeros((B,T,C)) # bag of words
for b in range(B):
  for t in range(T):
    xprev = x[b,:t+1] # (t,C), record the current step as previous step
    xbow[b,t] = torch.mean(xprev, 0) # take the average of all values

# this code takes the average over each 'time' dimension (rows) to record self-attention
# this can be done more efficiently without loops using matrix multiplication

In [15]:
# version 2: you can use a lower triangular matrix to do the same thing as the loop above!
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a@b # dot product
print(a)
print(b)
print(c)

# note that this is not the running average!

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[8., 6.],
        [5., 2.],
        [4., 4.]])
tensor([[ 8.,  6.],
        [13.,  8.],
        [17., 12.]])


In [16]:
# version 2: to get a historical running average, you need to average out the weights
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei@x # (B, T,T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2, atol=1e-4) # boolean if two tensors are equal to the given tolerance (flotas are annoying)

True

In [17]:
# version 3: you can also use softmax to calculate running average!
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf')) # set 0s to -inf to be processed by softmax ignoring future history
wei = F.softmax(wei, dim=-1) # torch.nn.functional.softmax -> normalizes out the -infinities
xbow3 = wei@x
torch.allclose(xbow, xbow3, atol=1e-4) # still equal, but more efficient!

True

In [18]:
# version 4: self-attention!
# self-attention allows the neural network to have positional encoding in addition to semantic encoding.
# This is similar to the principle that a CNN won't give location for object detection networks like YOLO.
# see the wei[0] printout below for interpretation of what weights are doing
# essentially it boils down to abuse of linear algebra, but the concept is more important than the math (feel free to review the last printout though).

# from version 3
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x=torch.randn(B,T,C)

# let's see a single head perform in self attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False) # we don't aggregate x, but rather x applied to a 'value' linaer layer
k = key(x) # (B, T, 16==head_size)
q = query(x) # (B, T, 16==head_size)
wei = q@k.transpose(-2, -1) # need to tranpose last 2 dimensions of the key, so switch (T,16) to (16,T)  | q@k == (B,T,16) @ (B,16,T) --> (B,T,T)
print(wei[0])

tril = torch.tril(torch.ones(T,T))
# wei=torch.zeros((T,T)) # weight inputs are no longer zeros, but rather are values from the query @ key dot product
wei=wei.masked_fill(tril==0, float('-inf')) # weights are still applied a upper triangle mask so that nodes from the future do not talk to nodes from the past
print(wei[0])
wei = F.softmax(wei, dim=-1) # comment out to see what softmax is doing
# essentially it normalizes the outputs of each row signifying each value's importance in that row
# for each time layer.
print(wei[0].sum(dim=1)) # notice the sum of each row equals to 1

# again, we aggregate a value linear layer rather than the actual inputs
v=value(x) # input 32 values from x, output 16 values from v
out=wei@v
print(f"The input shape x: {x.shape}, x is ran through a linear layer outputting v of shape {v.shape}.")
print(f"Weights of shape: {wei.shape} are applied to v of {v.shape},")
print(f"producing the final output of {out.shape}.")

tensor([[-1.7629, -1.3011,  0.5652,  2.1616, -1.0674,  1.9632,  1.0765, -0.4530],
        [-3.3334, -1.6556,  0.1040,  3.3782, -2.1825,  1.0415, -0.0557,  0.2927],
        [-1.0226, -1.2606,  0.0762, -0.3813, -0.9843, -1.4303,  0.0749, -0.9547],
        [ 0.7836, -0.8014, -0.3368, -0.8496, -0.5602, -1.1701, -1.2927, -1.0260],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,  0.8638,  0.3719,  0.9258],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,  1.4187,  1.2196],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,  0.8048],
        [-1.8044, -0.4126, -0.8306,  0.5899, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)
tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf, 

In [19]:
# as you can see, in the last T (time) row, 0.2297 was very meaningful to the final outputs
# 0.2423 & 0.2391! The network learns from this and extracts meaning from the relational significance
# as well as the positional significance!
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)



Notes:

    * Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
    * There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
    * Each example across batch dimension is of course processed completely independently and never "talk" to each other
    * In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
    * "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
    * "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below



In [20]:
# what is 'scaled' attention though?
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q@k.transpose(-2,-1)*head_size**-0.5
wei[0]

tensor([[ 1.9857, -1.7370,  0.4189,  0.2985, -0.5451,  0.4942, -0.7267, -0.4810],
        [-0.6151,  0.7704,  0.1215,  0.1193, -1.0559, -0.1234,  0.3918, -0.2687],
        [ 0.4511,  0.6600,  0.8736, -0.5065,  0.8595,  0.2483,  0.7095, -0.3241],
        [ 1.4350, -0.5599,  1.2163, -0.0813,  1.7313,  0.3421, -0.3146, -0.9178],
        [-2.0204,  1.8716, -1.1214, -0.1317, -0.4320,  0.8461,  1.0991,  1.8651],
        [ 1.0000,  0.5394,  0.9807, -0.0900,  0.7364,  1.3018,  1.4779,  1.2385],
        [ 1.0542, -0.5249,  0.1258, -0.0781,  0.8236, -1.0546,  0.3601, -0.5679],
        [ 0.2587,  0.1620,  0.6471,  0.2837,  1.2641,  0.3890, -0.6218, -0.4601]])

We need to add multiple attention since one channel to learn relationships between the self-attention matrices is not enough.
Simply pass in more heads to the transformer block instead of one.

In [37]:
# train the new model in bigram_v2.py with the self-attention
# !python '/content/drive/MyDrive/Data_Science/LLM-from-scratch/bigram_v2.py'
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('/content/drive/MyDrive/Data_Science/LLM-from-scratch/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5091, val loss 2.5060
step 300: train loss 2.4199, val loss 2.4337
step 400: train loss 2.3500, val loss 2.3563
step 500: train loss 2.2961, val loss 2.3126
step 600: train loss 2.2408, val loss 2.2501
step 700: train loss 2.2053, val loss 2.2187
step 800: train loss 2.1636, val loss 2.1870
step 900: train loss 2.1226, val loss 2.1483
step 1000: train loss 2.1017, val loss 2.1283
step 1100: train loss 2.0683, val loss 2.1174
step 1200: train loss 2.0376, val loss 2.0798
step 1300: train loss 2.0256, val loss 2.0645
step 1400: train loss 1.9919, val loss 2.0362
step 1500: train loss 1.9696, val loss 2.0304
step 1600: train loss 1.9625, val loss 2.0470
step 1700: train loss 1.9402, val loss 2.0119
step 1800: train loss 1.9085, val loss 1.9957
step 1900: train loss 1.9080, val loss 1.9869
step 2000: train loss 1.8834, val loss 1.9941
step 2100: train loss 1.

In [38]:
context = torch.zeros((1, 1), dtype=torch.long, device='cpu')
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


And thou to lesserve his his know'st broy by A towe than or fuch dight none worthy'st countinne, congess
our ire Iname's marriatate the entrity?

COMIOLUS:
Yet there me your let thy by courtary, own but I cannot, to
you.

MOth Osque, while and nett; pity, brow umput;
He betwered's prettedy if not you arter,
But woman furner his good me to ambled thy follows
Gents for you daying this distend and he but.

COMINANUS:
But you know wish the wear? whoe not to breave maste gate?
Not, now you read own. Lo-honour shoes
honordore vilibert.

ARTOS:
Nay, as Is theen, God
Were I saying cose
Will there's upon and tools.

HORSIO:
Pomfort life?
Whereform make comps hersed, my what away,
Go'st Your haste entens, and succe?

LORD RIARENCE:
Fies my like, wifch a my nobt.
And!
And ways. Whithing death.

CORIOLUMNO:
It must I have grawits.-
Ris Gomisty yor then thin dot this no all-donged,
But quarry the latter: Have me the betime twooke steed to blood
That his rysour grower-foldds: bnot Plond,
By that al

Finished the video!
https://www.youtube.com/watch?v=kCc8FmEb1nY